In [1]:
import pandas as pd
import blocksci
import numpy as np
import zarr
import networkx as nx

DIR_PARSED="/mnt/hdd_data/blockchain_parsed/"
cfg_file = f"/mnt/hdd_data/blockchain_parsed/bitcoin.cfg" # later
chain = blocksci.Blockchain(cfg_file)

# scope
this notebook goal is to replicate some results from `paint-it-black` as a sanity check.
the original paper analysed form block 1 to block 400.000

# 1: Number of addresses per darknet
here we want to replicate table 1 from `pait-it-black`: 
measure the number of addresses for each entity

In [2]:
df = pd.read_csv(f"/home/nvalla/paint-black/uniform_black/heur_2_data/ground_truth_clust_id.csv")

In [3]:
entities_c = df['entity'].value_counts()

In [4]:
print(entities_c)

EvolutionMarket           419915
SilkRoadMarketplace       372553
SilkRoad2Market           349836
AlphaBayMarket            196827
NucleusMarket             146281
AbraxasMarket             119065
PandoraOpenMarket          55357
SheepMarketplace           53639
BlackBankMarket            50878
MiddleEarthMarketplace     34049
BlueSkyMarketplace         18897
CannabisRoadMarket          2829
BabylonMarket               1471
GreenRoadMarket              965
Name: entity, dtype: int64


The number of addresses per entity is close enough to table 1 from [].

# 2: Number of input and output transactions per darkent
here we replicate table 2 from `paint-it-black`:
Table showing the number of transactions received
and transactions sent from and to each of the 14 darknets.

In [ ]:
l = [(chain.address_from_string(a).input_txes_count(), chain.address_from_string(a).output_txes_count()) for a in df.address]

_this method doesnt work: may be counting transactions more than once(if two different address are in the same transaction) per entity. Table 2 caption is not clear enough._

In [ ]:
df["input_txes_count"] = [a[0] for a in l]
df["output_txes_count"] = [a[1] for a in l]

In [ ]:
df.groupby(['entity'])["input_txes_count"].sum()

In [ ]:
df.groupby(['entity'])["output_txes_count"].sum()

# 3:
table 3 from paint-it-black

In [2]:
df1 = pd.read_csv(f"/home/nvalla/paint-black/uniform_black/heur_1_data/ground_truth_clust_id.csv")

In [21]:
a = df1.groupby(['entity'])["cluster_id"].nunique()

In [22]:
a

entity
AbraxasMarket              81685
AlphaBayMarket            117659
BabylonMarket               1153
BlackBankMarket            36022
BlueSkyMarketplace         13336
CannabisRoadMarket          2053
EvolutionMarket           284173
GreenRoadMarket              811
MiddleEarthMarketplace     25733
NucleusMarket             112226
PandoraOpenMarket          39257
SheepMarketplace           35125
SilkRoad2Market           219536
SilkRoadMarketplace       252664
Name: cluster_id, dtype: int64

# 4:
table 4 from paint-it-black

In [26]:
df2 = pd.read_csv(f"/home/nvalla/paint-black/uniform_black/heur_2_data/ground_truth_clust_id.csv")

In [27]:
b = df2.groupby(['entity'])["cluster_id"].nunique()

In [28]:
b

entity
AbraxasMarket              71421
AlphaBayMarket            103288
BabylonMarket                908
BlackBankMarket            31255
BlueSkyMarketplace         10640
CannabisRoadMarket          1585
EvolutionMarket           253379
GreenRoadMarket              635
MiddleEarthMarketplace     22508
NucleusMarket              99026
PandoraOpenMarket          33484
SheepMarketplace           31154
SilkRoad2Market           193932
SilkRoadMarketplace       229203
Name: cluster_id, dtype: int64

# number of clusters 
number of clusters black from the ground truth

In [2]:
clust_is_black = zarr.load("uniform_black/heur_2_data/cluster_is_black_ground_truth.zarr")
print(clust_is_black.sum())

119


In [3]:
clust_is_black_when = zarr.load("uniform_black/heur_2_data/cluster_is_black_final_day.zarr")

In [2]:
df = pd.read_csv("uniform_black/heur_2_data/diffusion_net_day.csv")

In [7]:
cm = blocksci.cluster.ClusterManager(DIR_PARSED + "bitcoin/heur_2/", chain)

In [24]:
add = chain.address_from_string("17DEJSAMHsTtsxZA1v6yu51xVwtFJ5ECfp")
clu = cm.cluster_with_address(add)  # cluster index 5

blocks = []
for t in add.input_txes:
    blocks.append(t.block_height)
    
b = chain.blocks[blocks[0]]
print(b.time)
date = '2015-08-26'

In [25]:
g = nx.read_graphml(f'{DIR_PARSED}bitcoin/heur_2_networks_day/{date}.graphml.bz2')

In [27]:
'5' in g.nodes

True

In [30]:
df.loc[df.date == date]

,date,no_old_black_user,no_new_black_user,no_active_black_user,no_cum_black_user,black2new_no_trx,black2new_value,black2new_no_link,black2black_no_trx,black2black_value,black2black_no_link,not_black_link
2426,2015-08-26,0,0,0,0,0,0,0,0,0,0,170794


# simulate ub_diffusion_net on one day network

In [ ]:
g = nx.read_graphml(f'{DIR_PARSED}bitcoin/heur_2_networks_day/{date}.graphml.bz2')

In [54]:
clust_is_black_ground = zarr.load(f"uniform_black/heur_2_data/cluster_is_black_ground_truth.zarr")
clust_is_black_ground_set = set([str(i) for i in range(len(clust_is_black_ground)) if clust_is_black_ground[i]])

In [75]:
new_black_nodes = set([])
old_black_nodes = clust_is_black_ground_set.intersection(g.nodes)
print(old_black_nodes)
clust_is_black_when = np.zeros(len(clust_is_black_ground), dtype=str)

black2new_no_trx = 0
black2new_value = 0
black2new_link = 0
black2black_no_trx = 0
black2black_value = 0
black2black_link = 0

s = 0
for black_node in old_black_nodes:
    # check out-neighbours of black node
    for e in g[black_node]:
        # if not already black, track it
        if e not in old_black_nodes:
            new_black_nodes.update([e])
            black2new_no_trx += g[black_node][e]['n_tx']
            black2new_value += g[black_node][e]['value']
            black2new_link += 1
        else:
            black2black_no_trx += g[black_node][e]['n_tx']
            black2black_value += g[black_node][e]['value']
            black2black_link += 1
        if clust_is_black_when[int(e)] == '':
            clust_is_black_when[int(e)] = date
            s +=1


{'5'}


In [76]:
print(s)
np.count_nonzero(clust_is_black_when)

26028


26028